<a href="https://colab.research.google.com/github/SakchaiSaehoei/China-Scholarship-Web-Scriping/blob/main/scholarship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# day 1
extract data form the website
# day 2
For mat data and write them into csv

In [ ]:
 url = 'https://www.cucas.edu.cn/china_scholarships?cat=&degree=' + str(major_count) + '&school=&program=&lang=&semester=&page='
        iterations = page_count(url)

        https://www.cucas.cn/china_scholarships/index/all_scholarship/all_cities/all_universities/all_degrees/all_languages/all_year/all_programs/0_0_0_0_0_0_0/page=1

IndentationError: ignored

In [ ]:
import csv
import requests
import bs4
import io
import time
import pandas as pd

In [ ]:
def extract(url):
    r = requests.get(url) #returns the HTML of the page, can be done through urlopen as well
    soup = bs4.BeautifulSoup(r.content)

    all_td = soup.findAll("td")
    # 0 : titles 1 : language 2: Start 3 : scholarship 4: to_pay

    universities = []
    universities_logo = []
    titles = all_td[0:len(all_td):6]
    languages = all_td[1:len(all_td):6]
    start = all_td[2:len(all_td):6]
    scholarship = all_td[3:len(all_td):6]
    to_pay = all_td[4:len(all_td):6]
    #
    # for language  in languages:
    #   print(language.get_text())
    # print(titles)
    """I still need to return name of the university of each program so I plan to go to the link to each program inorder to get the University """
    # print(titles[0].find_all('a')[0]['href'])
    for idx, link in  enumerate(titles) :
      href = titles[idx].find_all('a')[0]['href']
      fullLink = "https://www.cucas.cn/" + href
      program_info_url = requests.get(fullLink) #returns the HTML of the page, can be done through urlopen as well
      program_info = bs4.BeautifulSoup(program_info_url.content)
      university_name = program_info.find('h4').text
      universities.append(university_name)
      university_logo = program_info.find('div',{"class": "img"})
      university_logo_url =   "https://www.cucas.cn/" + university_logo('img')[0]['src']
      universities_logo.append(university_logo_url)
      print(university_logo_url)
    return titles, languages, start, scholarship, to_pay , universities , university_logo_url

In [ ]:
def page_count(url):
    url = url + str(1)
    r = requests.get(url) #returns the HTML of the page, can be done through urlopen as well
    soup = bs4.BeautifulSoup(r.content)
    # print(soup)
    page_numbers = soup.findAll("ul",{"class":"paginator"})
    total_schols = int(page_numbers[0].text[page_numbers[0].text.find(':')+1:])
    if (total_schols%20==0):
        iterations = total_schols/20
    else:
        iterations = int(total_schols/20)+1
    return(int(iterations))

In [ ]:
#
dict = {}
program_id=1
with open('test_new.csv', 'w', newline='',encoding="utf-8") as fp:
    a = csv.writer(fp, delimiter=',')
    a.writerows([['Program ID','University','Program','Level','Language','Start Date','Tuition Covered', 'Accomodation covered?', 'Living Expense Covered?','Tuition fees to pay','Original Tuition fee','Accomodation to pay','Living expense to pay', 'School Logo']])
    majors = ['non_degree','bachelor_degree','master_degree','doctor_degree','undefined']
    levels = ['Associate','Bachelor','Master','Phd','Non-Degree']

    for id , major in enumerate(majors):
        url = 'https://www.cucas.cn/china_scholarships/index/all_scholarship/all_cities/all_universities/' + str(major) + '/all_languages/all_year/all_programs/0_0_0_0_' + str(id+1) + '_0_0/page='
        iterations = page_count(url)
        level = levels[id]
        # loop for extract information in each page

        for page in range(1,iterations+1):
            url_new = url + str(page)
            # print(url_new)
            titles, languages, start, scholarship, to_pay, universities, universities_logo = extract(url_new)
            # print(titles)
          # The result of titles, languages...... are bs4.element.Tag in side the list of all titles in the page
            # loop for get into each list and write into the csv file "a" row by row
            """handel : scholarship into tutuition, accomodation , living expense """
            for i in range(0,len(titles)):
              program = titles[i].text
              language = languages[i].text
              start_date = start[i].text
              uinversity = universities[i]

              tuition_acc_living = scholarship[i].p.text
              tuition = tuition_acc_living[tuition_acc_living.find(':')+1:tuition_acc_living.find('\n')].strip()

              accomodation = tuition_acc_living[tuition_acc_living.find(':',tuition_acc_living.find('Accom'))+1:tuition_acc_living.find('\n',tuition_acc_living.find('Accom'))].strip()

              living = tuition_acc_living[tuition_acc_living.find(':',tuition_acc_living.find('Living'))+1:].strip()

              To_Pay = to_pay[i].p.text
              try:
                   orig_tuition = to_pay[i].p.span.text.strip()
              except:
                  orig_tuition=-1
              try:
                  tuition_to_pay = To_Pay[To_Pay.find(':')+1:To_Pay.find('\n',To_Pay.find(':')+1)].strip().replace(orig_tuition,'').strip()
              except:
                  tuition_to_pay = To_Pay[To_Pay.find(':')+1:To_Pay.find('\n', To_Pay.find(':'))].strip()
              try:
                  accomodation_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Acco'))+1:To_Pay.find('\n',To_Pay.find(':',To_Pay.find('Acco')))].strip()
              except:
                   accomodation_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Acc'))+1:To_Pay.find('\n',To_Pay.find('Acc'))].strip()
              try:
                   Living_Expense_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Expense'))+1:To_Pay.find('\n',To_Pay.find(':',To_Pay.find('Expense')))].strip()
              except:
                   Living_Expense_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Liv'))+1:To_Pay.find('\n',To_Pay.find('Liv'))].strip()

              data = [[program_id, uinversity, program,level, language, start_date,tuition,accomodation, living, tuition_to_pay, orig_tuition,accomodation_to_pay,Living_Expense_to_pay,universities_logo]]
              program_id+=1
              a.writerows(data)

print("Done")



https://www.cucas.cn//resource/new/images/school/logo/logo_1329.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_159.png
https://www.cucas.cn//resource/new/images/school/logo/logo_1333.png
https://www.cucas.cn//resource/new/images/school/logo/logo_1333.png
https://www.cucas.cn//resource/new/images/school/logo/logo_

In [ ]:
airbnb_data = pd.read_csv("test_new.csv")

# View the first 5 rows
airbnb_data.head()

from google.colab import files
files.download("test_new.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import psycopg2

In [ ]:
dict = {}
schol_id=0
with open('test_new.csv', 'w', newline='',encoding="utf-8") as fp:
    a = csv.writer(fp, delimiter=',')
    a.writerows([['School ID','University','Major','Level','Language','Start Date','Tuition Covered', 'Accommodation Covered', 'Living Expense Covered','Tuition Fees To Pay','Original Tuition Fee','Accommodation To Pay','Living Expense To Pay']])
    for major_count in range(1,5):
        url = 'https://www.cucas.edu.cn/china_scholarships?cat=&degree=' + str(major_count) + '&school=&program=&lang=&semester=&page='
        iterations = page_count(url)

        for j in range(1,iterations+1):
            url_new = url + str(j)
            titles, majors , languages, start, scholarship, to_pay = extract(url_new)
            for i in range(0,len(majors)):
                title = titles[i].p.text
                major = majors[i].p.text
                if major_count==1:
                    level = 'Non-Degree'
                elif major_count==2:
                    level = 'Bachelor'
                elif major_count==3:
                    level = 'Master'
                elif major_count==4:
                    level = 'Phd'
                language = languages[i].text
                start_date = start[i].text

                tuition_acc_living = scholarship[i].p.text
                tuition = tuition_acc_living[tuition_acc_living.find(':')+1:tuition_acc_living.find('\n')].strip()

                accomodation = tuition_acc_living[tuition_acc_living.find(':',tuition_acc_living.find('Accom'))+1:tuition_acc_living.find('\n',tuition_acc_living.find('Accom'))].strip()

                living = tuition_acc_living[tuition_acc_living.find(':',tuition_acc_living.find('Living'))+1:].strip()

                To_Pay = to_pay[i].p.text
                try:
                    orig_tuition = to_pay[i].p.span.text.strip()
                except:
                    orig_tuition=-1
                try:
                    tuition_to_pay = To_Pay[To_Pay.find(':')+1:To_Pay.find('\n',To_Pay.find(':')+1)].strip().replace(orig_tuition,'').strip()
                except:
                    tuition_to_pay = To_Pay[To_Pay.find(':')+1:To_Pay.find('\n', To_Pay.find(':'))].strip()

                try:
                    accomodation_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Acco'))+1:To_Pay.find('\n',To_Pay.find(':',To_Pay.find('Acco')))].strip()
                except:
                    accomodation_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Acc'))+1:To_Pay.find('\n',To_Pay.find('Acc'))].strip()
                try:
                    Living_Expense_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Expense'))+1:To_Pay.find('\n',To_Pay.find(':',To_Pay.find('Expense')))].strip()
                except:
                    Living_Expense_to_pay = To_Pay[To_Pay.find(':',To_Pay.find('Liv'))+1:To_Pay.find('\n',To_Pay.find('Liv'))].strip()

                schol_id+=1
                data = [[schol_id,title, major,level, language, start_date,tuition,accomodation, living, tuition_to_pay, orig_tuition,accomodation_to_pay,Living_Expense_to_pay]]
                a.writerows(data)
print('done')